```
Decimal pixel code srrrrccc

s slice 1-4
rrrr row number 1-2058
ccc is column 1-512
```

In [1]:
import numpy as np

In [2]:
def pix(n): 
    return np.array(range(1,n+1))

In [3]:
slice_base = 10000000
row_base = 1000

In [4]:
def row(n):
    return row_base*n+np.concatenate((
            1 * slice_base + pix(512),
            2 * slice_base + pix(512),
            3 * slice_base + pix(512),
            4 * slice_base + pix(512),
        ))

Alternate definition for just a horizontal gradient.

def row(n):
    return pix(2048)

In [5]:
image=np.array(list(map(row,range(1,2059))))

In [6]:
from astropy.io import fits

In [7]:
fits.writeto('electron_order_test.fits', image, None, overwrite=True)

In [8]:
electrons=fits.open('electron_order_test.fits')

In [9]:
from httm.fits_utilities.electron_flux_fits import *

In [10]:
import logging
logging.basicConfig(
    format="[%(asctime)s] %(levelname)s [%(name)s.%(funcName)s:%(lineno)d] %(message)s",
    datefmt="%H:%M:%S",
)

In [11]:
ideal_data = electron_flux_converter_from_fits('electron_order_test.fits')

[16:09:38] WARNING [httm.fits_utilities.header_tools.get_header_setting:106] Required FITS keyword not present: CAMNUM
[16:09:38] WARNING [httm.fits_utilities.header_tools.get_header_setting:106] Required FITS keyword not present: CCDNUM
[16:09:38] WARNING [httm.fits_utilities.header_tools.get_header_setting:106] Required FITS keyword not present: N_FRAMES


In [12]:
dark_columns = numpy.zeros(11)
dark_rows = numpy.zeros((20,512+22))

In [13]:
def geometric_row(n,slice):
    return numpy.concatenate((dark_columns,slice_base*(slice+1)+row_base*n+pix(512),dark_columns))

In [14]:
def geometric_slice(slice):
    return numpy.concatenate((map(lambda n:geometric_row(n,slice),range(1,2059)),dark_rows))

In [15]:
assert np.array_equal(geometric_slice(0),ideal_data.slices[0].pixels), \
           "Slice from FITS does not match expected code"

In [16]:
assert np.array_equal(numpy.fliplr(geometric_slice(1)),ideal_data.slices[1].pixels), \
           "Slice from FITS does not match expected code"

In [17]:
assert np.array_equal(geometric_slice(2),ideal_data.slices[2].pixels), \
           "Slice from FITS does not match expected code"

In [18]:
assert np.array_equal(numpy.fliplr(geometric_slice(3)),ideal_data.slices[3].pixels), \
           "Slice from FITS does not match expected code"

In [19]:
write_electron_flux_converter_to_simulated_raw_fits(ideal_data,"electrons_raw.fits")

In [20]:
munged_electrons=fits.open('electrons_raw.fits')

In [21]:
assert np.array_equal(munged_electrons[0].data[:-20,44:-44],electrons[0].data), \
"Reassembled FITS from electrons does not equal given electrons"